jupyter-lab --allow-root --NotebookApp.allow_origin='https://colab.research.google.com' --port=8888 --NotebookApp.port_retries=0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from PIL import Image
import os
import json
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import torch.optim as optim
import torchvision.transforms.functional as TF
import torch.nn.functional as F

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.image_files = os.listdir(self.image_dir)
        self.label_files = os.listdir(self.label_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image_filename = self.image_files[idx]
        image_path = os.path.join(self.image_dir, image_filename)
        label_filename = image_filename.replace(".tif", ".json")
        label_path = os.path.join(self.label_dir, label_filename)

        # Load image
        image = Image.open(image_path)

        # Read label file
        with open(label_path, 'r') as f:
            label_data = json.load(f)

        # Extract bounding box coordinates
        boxes = []
        for obj in label_data["objects"]:
            if obj["classTitle"] == "Oil tank":  # Assuming the class name for oil tanks is "Oil tank"
                bbox = obj["points"]["exterior"]
                x_min, y_min = bbox[0]
                x_max, y_max = bbox[1]
                boxes.append([x_min, y_min, x_max, y_max])

        # Convert to tensors
        image_tensor = ToTensor()(image)
        boxes_tensor = torch.tensor(boxes, dtype=torch.float32)

        # Labels are not used in Faster R-CNN, but we include them for consistency
        num_boxes = len(boxes)
        labels_tensor = torch.ones((num_boxes,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes_tensor
        target["labels"] = labels_tensor

        return image_tensor, target


In [ ]:
# Set the directory paths in your Google Drive
drive_path = "/content/drive/MyDrive/oiltank_dataset"
image_dir = os.path.join(drive_path, "train_images")
label_dir = os.path.join(drive_path, "train_labels")

In [ ]:
# Set the transformation
transform = None  # You can add additional transformations if needed

In [ ]:
# Create the custom dataset
dataset = CustomDataset(image_dir, label_dir, transform=transform)

In [ ]:
# Set batch size and create data loader
batch_size = 16
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# Train your Faster R-CNN model using the dataloader
model = fasterrcnn_resnet50_fpn(pretrained=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


In [ ]:
# Set the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

In [ ]:
# Set the number of epochs
num_epochs = 10

In [ ]:
def compute_loss(targets, outputs):
    # Extract the predicted labels and bounding boxes from the model outputs
    pred_labels = outputs["labels"]
    pred_boxes = outputs["boxes"]

    # Extract the ground truth labels and bounding boxes from the targets
    gt_labels = targets["labels"]
    gt_boxes = targets["boxes"]

    # Compute the classification loss
    classification_loss = F.cross_entropy(pred_labels, gt_labels)

    # Compute the regression loss
    regression_loss = F.smooth_l1_loss(pred_boxes, gt_boxes)

    # Combine the classification and regression losses
    loss = classification_loss + regression_loss

    return loss

In [ ]:
for epoch in range(num_epochs):
    for images, targets in dataloader:
        images = images.to(device)
        targets = {k: v.to(device) for k, v in targets.items()}

        # Forward pass
        outputs = model(images, targets)

        # Compute the loss
        loss = compute_loss(targets, outputs)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss every few iterations
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')